In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import tejapi
import yfinance as yf
import cufflinks as cf
pd.options.mode.chained_assignment = None
from pandas_datareader import data
tejapi.ApiConfig.api_key ="gJmwWQMfIUIuyCKhYsQD10Aa0wIQ8f"

In [95]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [96]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
tickers

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [97]:
tickers = [i.replace('.','-') for i in tickers]
tickers = tickers.Symbol.to_list()

AttributeError: 'list' object has no attribute 'Symbol'

In [ ]:
def RSIcalc(asset):
    df = yf.download(asset , start ="2011-01-01")
    df['MA200'] =  df['Adj Close'].rolling(window=200).mean()
    df['price change'] = df ['Adj Close'].pct_change()
    df['Upmove'] = df['price change'].apply(lambda x :x if x > 0 else 0)
    df['Downmove'] = df['price change'].apply(lambda x : abs(x) if x < 0 else 0)
    df['avg Up'] =df['Upmove'].ewm(span=19).mean() 
    df['avg Down'] =df['Downmove'].ewm(span=19).mean() 
    df = df.dropna()
    df['RS'] = df['avg Up']/df['avg Down']
    df['RSI'] = df['RS'].apply(lambda x:100-(100/(x+1)))
    df.loc[(df['Adj Close'] > df['MA200']) &(df['RSI'] <30),'Buy'] = 'Yes'
    df.loc[(df['Adj Close'] < df['MA200']) |(df['RSI'] >30),'Buy'] = 'NO'
    return df 
RSIcalc(tickers[0])

In [98]:
def getSignals(df):
    Buying_dates = []
    Selling_dates  = []
    
    for i in range(len(df)-11):
        if "Yes" in df['Buy'].iloc[i]:
            Buying_dates.append(df.iloc[i+1].name)
            for j in range(1, 11):
                if df["RSI"].iloc[i + j] >40:
                    Selling_dates.append(df.iloc[i+j+1].name)
                    break     
                elif j == 10:
                    Selling_dates.append(df.iloc[i+j+1].name)
    return Buying_dates, Selling_dates

In [99]:
frame = RSIcalc(tickers[0])
buy, sell = getSignals(frame)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- SYMBOL: No data found, symbol may be delisted


ValueError: cannot set a frame with no defined index and a scalar

In [100]:
tickers.Close

AttributeError: 'list' object has no attribute 'Close'